# Setup for google colab

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import sys
%cd "/content/drive/MyDrive/ColabNotebooks/Audio-ex3"
sys.path.append("/content/drive/MyDrive/ColabNotebooks/Audio-ex3")

In [ ]:
!git pull

In [ ]:
!pip install -r requirements.txt
%cd src

---
# Check if GPU is available

In [1]:
import torch
torch.cuda.is_available()

True

---
# Logger

In [1]:
from lightning.pytorch.loggers import NeptuneLogger
from dotenv import load_dotenv
import os

load_dotenv('./../.env')
API_TOKEN = os.environ.get("API_TOKEN")
PROJECT_NAME = os.environ.get("PROJECT_NAME")

logger_config = {
    "api_key": API_TOKEN,
    "project_name": PROJECT_NAME,
    "log_model_checkpoints": False
}

---
# Loading Data

In [19]:
import importlib
imported_module = importlib.import_module("loader")
importlib.reload(imported_module)
import loader
dm = loader.AudioDataModule()
dm.setup("fit")

# Create the classifier and train it

In [18]:
import importlib
importlib.reload(importlib.import_module("classifier"))
importlib.reload(importlib.import_module("loader"))
import classifier

neptune_logger = NeptuneLogger(project=PROJECT_NAME, api_key=API_TOKEN, log_model_checkpoints=False)
config = classifier.default_config

trainer = classifier.train_func(config=config, dm=dm, logger=neptune_logger)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/mrobay/Audio-ex3/e/AUD-124


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | conv1   | Conv1d  | 329 K  | train
1 | conv2   | Conv1d  | 327 K  | train
2 | linear  | Linear  | 378    | train
3 | relu    | ReLU    | 0      | train
4 | dropout | Dropout | 0      | train
5 | loss    | CTCLoss | 0      | train
--------------------------------------------
657 K     Trainable params
0         Non-trainable params
657 K     Total params
2.631     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 1 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/mrobay/Audio-ex3/e/AUD-124/metadata


# Validation

In [ ]:
import importlib
importlib.reload(importlib.import_module("classifier"))
importlib.reload(importlib.import_module("loader"))
importlib.reload(importlib.import_module("model_tuner"))
import model_tuner

model_tuner.tune_classifier(logger_config=logger_config, dm=dm)

2024-07-23 19:13:11,440	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


# Test the classifier

In [27]:
import importlib
importlib.reload(importlib.import_module("classifier"))
importlib.reload(importlib.import_module("loader"))
import classifier

trainer.test(model, data_loader['test'])

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

AttributeError: 'NeuralNetwork' object has no attribute 'argmax_prob'